In [1]:
from modules.db_connection import create_pg_engine
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# FUNÇÃO PRINCIPAL - DADOS INTEGRADOS COM JOIN INLINE
def load_complete_ride_data():
    """
    Carrega dados integrados IES + Cursos da RIDE-DF usando JOIN inline
    """
    try:
        engine = create_pg_engine()
        
        # Query completa com todos os JOINs necessários
        query = """
        SELECT 
            -- ===== dados das ies + geografia =====
            es."nu_ano_censo"::text as nu_ano_censo,
			es."co_municipio_ies"::text as co_municipio_ies,
			mun.nome_municipio,
			mun.municipio_capital,
			mun.longitude,
			mun.latitude,
			uf.sigla_uf,
			uf.nome_uf,
			reg.nome_regiao,
			es."in_capital_ies"::text as in_capital_ies,
			
			-- dados institucionais das ies
			es."co_ies"::text as co_ies,
			es."no_ies" as no_ies,
			es."sg_ies" as sg_ies,
			es."no_mantenedora" as no_mantenedora,
			es."tp_categoria_administrativa"::text as tp_categoria_administrativa,
			es."tp_rede"::text as tp_rede,
			es."tp_organizacao_academica"::text as tp_organizacao_academica,
			es."in_comunitaria"::text as in_comunitaria,
			es."in_confessional"::text as in_confessional,
			
			-- endereço das ies
			es."ds_endereco_ies" as ds_endereco_ies,
			es."no_bairro_ies" as no_bairro_ies,
			es."nu_cep_ies"::text as nu_cep_ies,
			
			-- recursos humanos das ies
			es."qt_doc_total"::integer as qt_doc_total,
			es."qt_doc_exe"::integer as qt_doc_exe,
			es."qt_doc_ex_femi"::integer as qt_doc_ex_femi,
			es."qt_doc_ex_masc"::integer as qt_doc_ex_masc,
			es."qt_doc_ex_grad"::integer as qt_doc_ex_grad,
			es."qt_doc_ex_esp"::integer as qt_doc_ex_esp,
			es."qt_doc_ex_mest"::integer as qt_doc_ex_mest,
			es."qt_doc_ex_dout"::integer as qt_doc_ex_dout,
			es."qt_doc_ex_int"::integer as qt_doc_ex_int,
			es."qt_doc_ex_parc"::integer as qt_doc_ex_parc,
			es."qt_doc_ex_hor"::integer as qt_doc_ex_hor,
			
			-- técnicos das ies
			es."qt_tec_total"::integer as qt_tec_total,
			es."qt_tec_superior_fem"::integer as qt_tec_superior_fem,
			es."qt_tec_superior_masc"::integer as qt_tec_superior_masc,
			
			-- infraestrutura digital das ies
			es."in_acesso_portal_capes"::text as in_acesso_portal_capes,
			es."in_repositorio_institucional"::text as in_repositorio_institucional,
			es."in_servico_internet"::text as in_servico_internet,
			es."in_catalogo_online"::text as in_catalogo_online,
			es."qt_periodico_eletronico"::integer as qt_periodico_eletronico,
			es."qt_livro_eletronico"::integer as qt_livro_eletronico,
			
			-- ===== dados dos cursos =====
			-- identificação dos cursos
			cur."co_curso"::text as co_curso,
			cur."no_curso" as no_curso,
			cur."co_cine_area_geral"::text as co_cine_area_geral,
			cur."no_cine_area_geral" as no_cine_area_geral,
			cur."co_cine_area_especifica"::text as co_cine_area_especifica,
			cur."no_cine_area_especifica" as no_cine_area_especifica,
			
			-- características dos cursos
			cur."tp_grau_academico"::text as tp_grau_academico,
			cur."in_gratuito"::text as in_gratuito,
			cur."tp_modalidade_ensino"::text as tp_modalidade_ensino,
			cur."tp_nivel_academico"::text as tp_nivel_academico,
			
			-- dados de vagas
			cur."qt_vg_total"::integer as qt_vg_total,
			cur."qt_vg_nova"::integer as qt_vg_nova,
			cur."qt_vg_proc_seletivo"::integer as qt_vg_proc_seletivo,
			
			-- dados de inscritos
			cur."qt_inscrito_total"::integer as qt_inscrito_total,
			cur."qt_insc_vg_nova"::integer as qt_insc_vg_nova,
			cur."qt_insc_proc_seletivo"::integer as qt_insc_proc_seletivo,
			
			-- essencial: dados de ingressos
			cur."qt_ing"::integer as qt_ing,
			cur."qt_ing_fem"::integer as qt_ing_fem,
			cur."qt_ing_masc"::integer as qt_ing_masc,
			cur."qt_ing_diurno"::integer as qt_ing_diurno,
			cur."qt_ing_noturno"::integer as qt_ing_noturno,
			cur."qt_ing_vestibular"::integer as qt_ing_vestibular,
			cur."qt_ing_enem"::integer as qt_ing_enem,
			
			-- essencial: dados de matrículas
			cur."qt_mat"::integer as qt_mat,
			cur."qt_mat_fem"::integer as qt_mat_fem,
			cur."qt_mat_masc"::integer as qt_mat_masc,
			cur."qt_mat_diurno"::integer as qt_mat_diurno,
			cur."qt_mat_noturno"::integer as qt_mat_noturno,
			
			-- essencial: dados de conclusões (para taxa de conclusão)
			cur."qt_conc"::integer as qt_conc,
			cur."qt_conc_fem"::integer as qt_conc_fem,
			cur."qt_conc_masc"::integer as qt_conc_masc,
			
			-- demografia estudantil - idade
			cur."qt_ing_18_24"::integer as qt_ing_18_24,
			cur."qt_ing_25_29"::integer as qt_ing_25_29,
			cur."qt_ing_30_34"::integer as qt_ing_30_34,
			cur."qt_mat_18_24"::integer as qt_mat_18_24,
			cur."qt_mat_25_29"::integer as qt_mat_25_29,
			cur."qt_mat_30_34"::integer as qt_mat_30_34,
			
			-- demografia estudantil - raça/cor
			cur."qt_ing_branca"::integer as qt_ing_branca,
			cur."qt_ing_preta"::integer as qt_ing_preta,
			cur."qt_ing_parda"::integer as qt_ing_parda,
			cur."qt_ing_amarela"::integer as qt_ing_amarela,
			cur."qt_ing_indigena"::integer as qt_ing_indigena,
			cur."qt_mat_branca"::integer as qt_mat_branca,
			cur."qt_mat_preta"::integer as qt_mat_preta,
			cur."qt_mat_parda"::integer as qt_mat_parda,
			
			-- financiamento estudantil
			cur."qt_ing_financ"::integer as qt_ing_financ,
			cur."qt_ing_fies"::integer as qt_ing_fies,
			cur."qt_ing_prounii"::integer as qt_ing_prounii,
			cur."qt_ing_prounip"::integer as qt_ing_prounip,
			cur."qt_mat_financ"::integer as qt_mat_financ,
			cur."qt_mat_fies"::integer as qt_mat_fies,
			cur."qt_mat_prounii"::integer as qt_mat_prounii,
			cur."qt_mat_prounip"::integer as qt_mat_prounip,
			
			-- reserva de vagas (cotas)
			cur."qt_ing_reserva_vaga"::integer as qt_ing_reserva_vaga,
			cur."qt_ing_rvetnico"::integer as qt_ing_rvetnico,
			cur."qt_mat_reserva_vaga"::integer as qt_mat_reserva_vaga,
			cur."qt_mat_rvetnico"::integer as qt_mat_rvetnico
			
		from ed_superior_ies es
			join municipio_ride_brasilia ride on es."co_municipio_ies"::bpchar = ride.codigo_municipio_dv
			join municipio mun on ride.codigo_municipio_dv = mun.codigo_municipio_dv
			join unidade_federacao uf on mun.cd_uf = uf.cd_uf
			join regiao reg on uf.cd_regiao = reg.cd_regiao
			left join ed_superior_cursos cur on (
			es."co_ies" = cur."co_ies" and 
			es."nu_ano_censo" = cur."nu_ano_censo"
			)
		ORDER BY es."nu_ano_censo" desc, mun.nome_municipio, es."no_ies", cur."no_curso"
		"""
        
        with engine.connect() as conn:
            df = pd.read_sql(query, conn)
        
        return df, None
    except Exception as e:
        return None, str(e)

# Função para calcular métricas derivadas
def calcular_metricas_educacionais(df):
    """Calcula métricas derivadas dos dados integrados"""
    if df is None or df.empty:
        return df
    
    df_metrics = df.copy()
    
    # Taxa de conclusão (principal variável dependente)
    df_metrics['taxa_conclusao'] = np.where(
        df_metrics['qt_mat'] > 0,
        (df_metrics['qt_conc'] / df_metrics['qt_mat'] * 100).round(2),
        0
    )
    
    # Taxa de ingresso
    df_metrics['taxa_ingresso'] = np.where(
        df_metrics['qt_vg_total'] > 0,
        (df_metrics['qt_ing'] / df_metrics['qt_vg_total'] * 100).round(2),
        0
    )
    
    # Percentual feminino
    df_metrics['perc_feminino'] = np.where(
        df_metrics['qt_mat'] > 0,
        (df_metrics['qt_mat_fem'] / df_metrics['qt_mat'] * 100).round(1),
        0
    )
    
    # Percentual de doutores (IES)
    df_metrics['perc_doutores'] = np.where(
        df_metrics['qt_doc_total'] > 0,
        (df_metrics['qt_doc_ex_dout'] / df_metrics['qt_doc_total'] * 100).round(1),
        0
    )
    
    # Relação candidato/vaga
    df_metrics['relacao_candidato_vaga'] = np.where(
        df_metrics['qt_vg_total'] > 0,
        (df_metrics['qt_inscrito_total'] / df_metrics['qt_vg_total']).round(2),
        0
    )
    
    return df_metrics


In [ ]:
df, error = load_complete_ride_data()

In [28]:
df

,nu_ano_censo,co_municipio_ies,nome_municipio,municipio_capital,longitude,latitude,sigla_uf,nome_uf,nome_regiao,in_capital_ies,...,qt_ing_prounii,qt_ing_prounip,qt_mat_financ,qt_mat_fies,qt_mat_prounii,qt_mat_prounip,qt_ing_reserva_vaga,qt_ing_rvetnico,qt_mat_reserva_vaga,qt_mat_rvetnico
0,2023,5200258,Águas Lindas de Goiás,Não,-48.283444,-15.737939,GO,Goiás,Centro-Oeste,Não,...,0,0,0,0,0,0,1,0,1,0
1,2023,5200258,Águas Lindas de Goiás,Não,-48.283444,-15.737939,GO,Goiás,Centro-Oeste,Não,...,0,0,1,0,0,0,0,0,0,0
2,2023,5200258,Águas Lindas de Goiás,Não,-48.283444,-15.737939,GO,Goiás,Centro-Oeste,Não,...,0,0,1,0,0,0,0,0,0,0
3,2023,5200258,Águas Lindas de Goiás,Não,-48.283444,-15.737939,GO,Goiás,Centro-Oeste,Não,...,0,0,1,0,0,0,0,0,0,0
4,2023,5200258,Águas Lindas de Goiás,Não,-48.283444,-15.737939,GO,Goiás,Centro-Oeste,Não,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7346,2023,5221858,Valparaíso de Goiás,Não,-47.984111,-16.069575,GO,Goiás,Centro-Oeste,Não,...,0,0,57,6,8,6,0,0,0,0
7347,2023,5221858,Valparaíso de Goiás,Não,-47.984111,-16.069575,GO,Goiás,Centro-Oeste,Não,...,0,0,43,3,4,3,0,0,0,0
7348,2023,5221858,Valparaíso de Goiás,Não,-47.984111,-16.069575,GO,Goiás,Centro-Oeste,Não,...,0,0,0,0,0,0,0,0,0,0
7349,2023,5221858,Valparaíso de Goiás,Não,-47.984111,-16.069575,GO,Goiás,Centro-Oeste,Não,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# === 1. Carregar dados ===
df = pd.read_csv("censo_ride_df_completo.csv", low_memory=False)

# Padronizar colunas
df.columns = [c.lower() for c in df.columns]

# === 2. Preparar variáveis ===
# Total de docentes
df["qt_doc_total_calc"] = (
    df["qt_doc_ex_grad"].fillna(0)
    + df["qt_doc_ex_esp"].fillna(0)
    + df["qt_doc_ex_mest"].fillna(0)
    + df["qt_doc_ex_dout"].fillna(0)
)

# Proporções de docentes
df["prop_doc_grad"] = df["qt_doc_ex_grad"] / df["qt_doc_total_calc"].replace(0, np.nan)
df["prop_doc_esp"]  = df["qt_doc_ex_esp"]  / df["qt_doc_total_calc"].replace(0, np.nan)
df["prop_doc_mest"] = df["qt_doc_ex_mest"] / df["qt_doc_total_calc"].replace(0, np.nan)
df["prop_doc_dout"] = df["qt_doc_ex_dout"] / df["qt_doc_total_calc"].replace(0, np.nan)

# Proporções de ingressantes
df["prop_ing_brancos"] = df["qt_ing_branca"] / df["qt_ing"].replace(0, np.nan)
df["prop_ing_pretos"]  = df["qt_ing_preta"]  / df["qt_ing"].replace(0, np.nan)
df["prop_ing_pardos"]  = df["qt_ing_parda"]  / df["qt_ing"].replace(0, np.nan)

# Agrupar pretos + pardos
df["prop_ing_pp"] = (df["qt_ing_preta"] + df["qt_ing_parda"]) / df["qt_ing"].replace(0, np.nan)

# Proporção de financiados (FIES + PROUNI)
df["prop_ing_financiados"] = (
    df["qt_ing_fies"] + df["qt_ing_prounii"] + df["qt_ing_prounip"]
) / df["qt_ing"].replace(0, np.nan)

# === 3. Tratar NA e limites ===
# Substituir NaN por 0 nas proporções (quando qt_ing=0 ou qt_doc_total=0)
for col in ["prop_doc_grad","prop_doc_esp","prop_doc_mest","prop_doc_dout",
            "prop_ing_brancos","prop_ing_pretos","prop_ing_pardos",
            "prop_ing_pp","prop_ing_financiados"]:
    df[col] = df[col].fillna(0).clip(0,1)

# Remover linhas sem variável dependente
df_model = df[df["qt_ing"].notnull() & (df["qt_ing"] >= 0)].copy()

# === 4. Fórmula do modelo ===
formula = """
qt_ing ~ C(tp_rede)
       + C(tp_organizacao_academica)
       + C(tp_grau_academico)
       + C(tp_modalidade_ensino)
       + qt_mat
       + qt_conc
       + prop_doc_mest
       + prop_doc_dout
       + prop_ing_pp
       + prop_ing_financiados
"""

# === 5. Ajustar Regressão Binomial Negativa ===
modelo_nb = smf.glm(
    formula=formula,
    data=df_model,
    family=sm.families.NegativeBinomial()
).fit()

print(modelo_nb.summary())

# === 6. Razões de taxa (IRR) ===
irr = np.exp(modelo_nb.params)
print("\n=== Razões de Taxa (IRR) ===")
print(irr)


/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:                 qt_ing   No. Observations:                 7351
Model:                            GLM   Df Residuals:                     7336
Model Family:        NegativeBinomial   Df Model:                           14
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -18031.
Date:                qui, 18 set 2025   Deviance:                       9692.3
Time:                        10:58:48   Pearson chi2:                 1.57e+04
No. Iterations:                    58   Pseudo R-squ. (CS):             0.8930
Covariance Type:            nonrobust                                         
                                                                                         coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------

In [32]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# === 1. Carregar dados ===
df = pd.read_csv("censo_ride_df_completo.csv", low_memory=False)
df.columns = [c.lower() for c in df.columns]

# === 2. Preparar variáveis ===
# Total de docentes
df["qt_doc_total_calc"] = (
    df["qt_doc_ex_grad"].fillna(0)
    + df["qt_doc_ex_esp"].fillna(0)
    + df["qt_doc_ex_mest"].fillna(0)
    + df["qt_doc_ex_dout"].fillna(0)
)

# Proporção de docentes com titulação avançada (mestres + doutores)
df["prop_doc_avancado"] = (
    (df["qt_doc_ex_mest"].fillna(0) + df["qt_doc_ex_dout"].fillna(0))
    / df["qt_doc_total_calc"].replace(0, np.nan)
)

# Proporções de ingressantes
df["prop_ing_pp"] = (df["qt_ing_preta"] + df["qt_ing_parda"]) / df["qt_ing"].replace(0, np.nan)
df["prop_ing_financiados"] = (
    df["qt_ing_fies"] + df["qt_ing_prounii"] + df["qt_ing_prounip"]
) / df["qt_ing"].replace(0, np.nan)

# Tratar NaN e limites
for col in ["prop_doc_avancado","prop_ing_pp","prop_ing_financiados"]:
    df[col] = df[col].fillna(0).clip(0,1)

# === 3. Filtrar observações válidas ===
df_model = df[(df["qt_ing"].notnull()) & (df["qt_ing"] >= 0) & (df["qt_mat"] > 0)].copy()

# Criar offset
df_model["offset_log_qtmat"] = np.log(df_model["qt_mat"])

# === 4. Fórmula ===
formula = """
qt_ing ~ C(tp_rede)
       + C(tp_organizacao_academica)
       + C(tp_grau_academico)
       + C(tp_modalidade_ensino)
       + qt_conc
       + prop_doc_avancado
       + prop_ing_pp
       + prop_ing_financiados
"""

# === 5. Ajustar modelo NB com offset ===
modelo_nb2 = smf.glm(
    formula=formula,
    data=df_model,
    family=sm.families.NegativeBinomial(),
    offset=df_model["offset_log_qtmat"]
).fit()

print(modelo_nb2.summary())

# === 6. Razões de taxa (IRR) ===
irr2 = np.exp(modelo_nb2.params)
print("\n=== Razões de Taxa (IRR) ===")
print(irr2)


/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:                 qt_ing   No. Observations:                 5427
Model:                            GLM   Df Residuals:                     5414
Model Family:        NegativeBinomial   Df Model:                           12
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -13401.
Date:                qui, 18 set 2025   Deviance:                       2806.2
Time:                        11:03:11   Pearson chi2:                 2.26e+03
No. Iterations:                   100   Pseudo R-squ. (CS):             0.1003
Covariance Type:            nonrobust                                         
                                                                                         coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------

In [25]:
# value_counts nas variáveis: tp_rede, tp_organizacao_academica
df['tp_nivel_academico'].value_counts()


tp_nivel_academico
Graduação    7351
Name: count, dtype: int64

In [4]:
df.columns

Index(['nu_ano_censo', 'co_municipio_ies', 'nome_municipio',
       'municipio_capital', 'longitude', 'latitude', 'sigla_uf', 'nome_uf',
       'nome_regiao', 'in_capital_ies', 'co_ies', 'no_ies', 'sg_ies',
       'no_mantenedora', 'tp_categoria_administrativa', 'tp_rede',
       'tp_organizacao_academica', 'in_comunitaria', 'in_confessional',
       'ds_endereco_ies', 'no_bairro_ies', 'nu_cep_ies', 'qt_doc_total',
       'qt_doc_exe', 'qt_doc_ex_femi', 'qt_doc_ex_masc', 'qt_doc_ex_grad',
       'qt_doc_ex_esp', 'qt_doc_ex_mest', 'qt_doc_ex_dout', 'qt_doc_ex_int',
       'qt_doc_ex_parc', 'qt_doc_ex_hor', 'qt_tec_total',
       'qt_tec_superior_fem', 'qt_tec_superior_masc', 'in_acesso_portal_capes',
       'in_repositorio_institucional', 'in_servico_internet',
       'in_catalogo_online', 'qt_periodico_eletronico', 'qt_livro_eletronico',
       'co_curso', 'no_curso', 'co_cine_area_geral', 'no_cine_area_geral',
       'co_cine_area_especifica', 'no_cine_area_especifica',
       'tp_gra

In [5]:
# Tabela com Cursos com mais Financiamentos (FIES, ProUni)
df_financiamento = df[['no_curso', 'no_ies', 'qt_mat_fies', 'qt_mat_prounii', 'qt_mat_prounip']].copy()
df_financiamento['total_financiamento'] = df_financiamento[['qt_mat_fies', 'qt_mat_prounii', 'qt_mat_prounip']].sum(axis=1)
df_financiamento = df_financiamento.sort_values(by='total_financiamento', ascending=False).head(20)
df_financiamento

,no_curso,no_ies,qt_mat_fies,qt_mat_prounii,qt_mat_prounip,total_financiamento
616,Medicina Veterinária,CENTRO UNIVERSITÁRIO DO PLANALTO CENTRAL APPAR...,106,90,90,286
117,Direito,CENTRO UNIVERSITÁRIO DO DISTRITO FEDERAL,53,215,11,279
5427,Psicologia,CENTRO UNIVERSITÁRIO EURO-AMERICANO,0,25,233,258
136,Odontologia,CENTRO UNIVERSITÁRIO DO DISTRITO FEDERAL,123,114,9,246
6661,Direito,UNIVERSIDADE CATÓLICA DE BRASÍLIA,98,145,2,245
138,Psicologia,CENTRO UNIVERSITÁRIO DO DISTRITO FEDERAL,33,185,5,223
601,Direito,CENTRO UNIVERSITÁRIO DO PLANALTO CENTRAL APPAR...,47,40,136,223
120,Enfermagem,CENTRO UNIVERSITÁRIO DO DISTRITO FEDERAL,76,134,3,213
5398,Direito,CENTRO UNIVERSITÁRIO EURO-AMERICANO,0,27,186,213
618,Odontologia,CENTRO UNIVERSITÁRIO DO PLANALTO CENTRAL APPAR...,79,48,62,189


In [9]:
# Remover duplicações por IES
df_docentes = df.drop_duplicates(subset=['co_ies'])

# Agora sim, somar os docentes executores do DF
total_docentes_df = df_docentes[df_docentes['sigla_uf'] == 'DF']['qt_doc_exe'].sum()
print(total_docentes_df)

8530


In [10]:
# Agregar os docentes por instituição
df_docentes_agg = df.groupby(['co_ies', 'sigla_uf']).agg({
    'qt_doc_exe': 'first',
    'qt_doc_ex_dout': 'first',
    'qt_doc_ex_mest': 'first',
    'qt_doc_ex_esp': 'first'
}).reset_index()

# Agora calcular apenas para DF
df_docentes_agg[df_docentes_agg['sigla_uf'] == 'DF']['qt_doc_exe'].sum()

np.int64(8530)

In [11]:
# Procura por UNB
df_unb = df[df['sg_ies'] == 'UNB']
df_unb['no_curso'].unique()
df_unb

,nu_ano_censo,co_municipio_ies,nome_municipio,municipio_capital,longitude,latitude,sigla_uf,nome_uf,nome_regiao,in_capital_ies,...,qt_ing_prounii,qt_ing_prounip,qt_mat_financ,qt_mat_fies,qt_mat_prounii,qt_mat_prounip,qt_ing_reserva_vaga,qt_ing_rvetnico,qt_mat_reserva_vaga,qt_mat_rvetnico
7030,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,13,1,10,2
7031,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,67,20,65,17
7032,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,24,14,23,14
7033,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,203,47,215,55
7034,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,14,4,13,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7223,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,0,0,1,1
7224,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,0,0,1,1
7225,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,2,1,59,19
7226,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,20,5,208,89


In [13]:
# Constrói um DataFrame com a quantidade de IES por município da RIDE-DF, separado por pública e privada. Mostra o valor total
df_ies_municipio = df[['co_municipio_ies', 'nome_municipio', 'tp_categoria_administrativa']].drop_duplicates()
df_ies_count = df_ies_municipio.groupby(['co_municipio_ies', 'nome_municipio', 'tp_categoria_administrativa']).size().reset_index(name='count')
df_ies_pivot = df_ies_count.pivot_table(index=['co_municipio_ies', 'nome_municipio'], columns='tp_categoria_administrativa', values='count', fill_value=0).reset_index()
df_ies_pivot['total_ies'] = df_ies_pivot.sum(axis=1, numeric_only=True)
df_ies_pivot

tp_categoria_administrativa,co_municipio_ies,nome_municipio,Privada com fins lucrativos,Privada sem fins lucrativos,Pública Estadual,Pública Federal,total_ies
0,3170404,Unaí,1.0,1.0,0.0,0.0,2.0
1,5200258,Águas Lindas de Goiás,1.0,0.0,0.0,0.0,1.0
2,5205497,Cidade Ocidental,1.0,0.0,0.0,0.0,1.0
3,5206206,Cristalina,1.0,0.0,0.0,0.0,1.0
4,5208004,Formosa,1.0,0.0,0.0,0.0,1.0
5,5208608,Goianésia,0.0,1.0,1.0,0.0,2.0
6,5212501,Luziânia,1.0,0.0,0.0,0.0,1.0
7,5215231,Novo Gama,1.0,0.0,0.0,0.0,1.0
8,5215603,Padre Bernardo,0.0,1.0,0.0,0.0,1.0
9,5221858,Valparaíso de Goiás,1.0,0.0,0.0,0.0,1.0


In [16]:
df['in_gratuito'].value_counts()

in_gratuito
Não    7105
Sim     246
Name: count, dtype: int64

In [17]:
# variáveis dentro de tp_modalidade_ensino
df['tp_modalidade_ensino'].value_counts()

tp_modalidade_ensino
Curso a distância    6465
Presencial            886
Name: count, dtype: int64

In [18]:
# QUANTIDADE DE INGRESSANTES EM CURSOS A DISTÂNCIA
df_distancia = df[df['tp_modalidade_ensino'] == 'Curso a distância']
total_ingressantes_distancia = df_distancia['qt_ing'].sum()
total_ingressantes = df['qt_ing'].sum()
percentual_distancia = (total_ingressantes_distancia / total_ingressantes * 100).round(2)
print(f"Total de Ingressantes em Cursos a Distância: {total_ingressantes_distancia}")
print(f"Total de Ingressantes em Todos os Cursos: {total_ingressantes}")
print(f"Percentual de Ingressantes em Cursos a Distância: {percentual_distancia}%")
df_distancia

Total de Ingressantes em Cursos a Distância: 43581
Total de Ingressantes em Todos os Cursos: 92421
Percentual de Ingressantes em Cursos a Distância: 47.15%


,nu_ano_censo,co_municipio_ies,nome_municipio,municipio_capital,longitude,latitude,sigla_uf,nome_uf,nome_regiao,in_capital_ies,...,qt_mat_prounii,qt_mat_prounip,qt_ing_reserva_vaga,qt_ing_rvetnico,qt_mat_reserva_vaga,qt_mat_rvetnico,qt_doc_exe_safe,qt_vg_nova_safe,prop_doc_doutores,candidato_vaga
17,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,751,1,0.319574,0.000
19,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,751,1000,0.319574,0.278
21,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,751,1000,0.319574,0.580
23,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,751,1,0.319574,0.000
26,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,0,0,751,1,0.319574,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7224,2023,5300108,Brasília,Sim,-47.887905,-15.794087,DF,Distrito Federal,Centro-Oeste,Sim,...,0,0,0,0,1,1,2801,1,0.910389,0.000
7300,2023,5212501,Luziânia,Não,-47.955720,-16.258541,GO,Goiás,Centro-Oeste,Não,...,0,0,0,0,0,0,6,100,0.000000,0.200
7301,2023,5212501,Luziânia,Não,-47.955720,-16.258541,GO,Goiás,Centro-Oeste,Não,...,0,0,0,0,0,0,6,1,0.000000,0.000
7302,2023,5212501,Luziânia,Não,-47.955720,-16.258541,GO,Goiás,Centro-Oeste,Não,...,0,0,0,0,0,0,6,1,0.000000,0.000


In [6]:
# MODELO FREQUENTISTA - REGRESSÃO BINOMIAL NEGATIVA

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Supondo que seu DataFrame se chama 'df'
# df = pd.read_csv('seu_arquivo_filtrado_ride.csv')

# --- Preparação e Feature Engineering ---

# Tratar possíveis valores nulos ou infinitos que possam surgir
df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Você pode optar por preencher NaNs com 0 ou a mediana, dependendo da variável
# Ex: df['candidato_vaga'].fillna(0, inplace=True)


# 1. Criar variáveis de proporção e razão
# Evitar divisão por zero
df['qt_doc_exe_safe'] = df['qt_doc_exe'].replace(0, 1)
df['qt_vg_nova_safe'] = df['qt_vg_nova'].replace(0, 1)

df['prop_doc_doutores'] = df['qt_doc_ex_dout'] / df['qt_doc_exe_safe']
df['candidato_vaga'] = df['qt_insc_vg_nova'] / df['qt_vg_nova_safe']


# 2. Filtrar para a análise (ex: cursos que tiveram pelo menos 1 inscrito)
df_modelo = df[df['qt_insc_vg_nova'] > 0].copy()


# --- Modelagem ---

# 3. Definir a fórmula com as variáveis de OFERTA
# Note que a variável dependente agora é qt_ing
formula_ingressantes = """
    qt_ing ~ 
    candidato_vaga +
    prop_doc_doutores +
    in_capital_ies +
    in_gratuito +
    C(tp_modalidade_ensino) +
    C(tp_categoria_administrativa) +
    C(tp_grau_academico)
"""
# qt_vg_nova e qt_insc_vg_nova são bons preditores, mas `candidato_vaga` já os combina.
# Teste diferentes combinações para ver o que funciona melhor.

# 4. Criar e treinar o modelo de Regressão Binomial Negativa
modelo_ingressantes = smf.glm(formula=formula_ingressantes, 
                              data=df_modelo, 
                              family=sm.families.NegativeBinomial()).fit()

# 5. Analisar os resultados detalhados
print(modelo_ingressantes.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 qt_ing   No. Observations:                  997
Model:                            GLM   Df Residuals:                      986
Model Family:        NegativeBinomial   Df Model:                           10
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3662.3
Date:                qui, 18 set 2025   Deviance:                       1198.0
Time:                        10:00:51   Pearson chi2:                 1.07e+03
No. Iterations:                    29   Pseudo R-squ. (CS):             0.9124
Covariance Type:            nonrobust                                         
                                                                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------

/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [7]:
import pandas as pd
print(pd.crosstab(df_modelo['in_gratuito'], df_modelo['tp_modalidade_ensino']))

tp_modalidade_ensino  Curso a distância  Presencial
in_gratuito                                        
Não                                 279         571
Sim                                   7         140


In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Assumimos que seu DataFrame original se chama 'df'
# Se ele tiver outro nome, apenas ajuste a variável abaixo.
# Ex: df = pd.read_csv('seu_censo_ride.csv')

# --- 1. PREPARAÇÃO E ENGENHARIA DE ATRIBUTOS ---

# Garantir que não há valores infinitos que possam ter surgido de cálculos anteriores
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Criação de denominadores seguros para evitar divisão por zero
df['qt_doc_exe_safe'] = df['qt_doc_exe'].replace(0, 1)
df['qt_vg_nova_safe'] = df['qt_vg_nova'].replace(0, 1)

# Criação das variáveis explicativas para o modelo
df['prop_doc_doutores'] = df['qt_doc_ex_dout'] / df['qt_doc_exe_safe']
df['candidato_vaga'] = df['qt_insc_vg_nova'] / df['qt_vg_nova_safe']

# Filtrar para a análise (ex: cursos que tiveram pelo menos 1 inscrito)
# Usamos .copy() para evitar SettingWithCopyWarning
df_modelo = df[df['qt_insc_vg_nova'] > 0].copy()


# --- 2. CONSTRUÇÃO E TREINAMENTO DO MODELO REVISADO ---

print("---" * 20)
print(" EXECUTANDO O MODELO DE REGRESSÃO REVISADO (SEM 'in_gratuito')")
print("---" * 20)

# Fórmula revisada sem 'in_gratuito' para resolver a multicolinearidade
formula_ingressantes_revisada = """
    qt_ing ~ 
    candidato_vaga +
    prop_doc_doutores +
    in_capital_ies +
    C(tp_modalidade_ensino) +
    C(tp_categoria_administrativa) +
    C(tp_grau_academico)
"""

# Criação e treinamento do modelo de Regressão Binomial Negativa
modelo_ingressantes_revisado = smf.glm(formula=formula_ingressantes_revisada, 
                                       data=df_modelo, 
                                       family=sm.families.NegativeBinomial()).fit()


# --- 3. SUMÁRIO DO MODELO ---

print(modelo_ingressantes_revisado.summary())


# --- 4. ANÁLISES ADICIONAIS (PERFIL DOS INGRESSANTES) ---

print("\n\n")
print("---" * 20)
print(" EXECUTANDO A ANÁLISE DE PERFIL DOS INGRESSANTES")
print("---" * 20)

# Criar denominador seguro para as proporções de ingressantes
df_modelo['qt_ing_safe'] = df_modelo['qt_ing'].replace(0, 1)

# Calcular as proporções de perfil baseadas nos ingressantes
df_modelo['prop_ing_mulheres'] = df_modelo['qt_ing_fem'] / df_modelo['qt_ing_safe']
df_modelo['prop_ing_prouni'] = (df_modelo['qt_ing_prounii'] + df_modelo['qt_ing_prounip']) / df_modelo['qt_ing_safe']
df_modelo['prop_ing_fies'] = df_modelo['qt_ing_fies'] / df_modelo['qt_ing_safe']
df_modelo['prop_ing_noturno'] = df_modelo['qt_ing_noturno'] / df_modelo['qt_ing_safe']
df_modelo['prop_ing_pardos'] = df_modelo['qt_ing_parda'] / df_modelo['qt_ing_safe']
df_modelo['prop_ing_pretos'] = df_modelo['qt_ing_preta'] / df_modelo['qt_ing_safe']
df_modelo['prop_ing_negros'] = df_modelo['prop_ing_pardos'] + df_modelo['prop_ing_pretos'] # Somando pardos e pretos

# --- 5. AGRUPAMENTO E VISUALIZAÇÃO DO PERFIL ---

# Agrupar por categoria administrativa e modalidade para ver a média de cada perfil
perfil_agrupado = df_modelo.groupby(['tp_categoria_administrativa', 'tp_modalidade_ensino']).agg(
    media_prop_mulheres=('prop_ing_mulheres', 'mean'),
    media_prop_negros=('prop_ing_negros', 'mean'),
    media_prop_noturno=('prop_ing_noturno', 'mean'),
    media_prop_prouni=('prop_ing_prouni', 'mean'),
    media_prop_fies=('prop_ing_fies', 'mean'),
    numero_de_cursos=('co_curso', 'count')
)

# Arredondar os resultados para melhor visualização
perfil_agrupado = perfil_agrupado.round(2)

print("Perfil médio dos ingressantes por tipo de IES e Modalidade na RIDE-DF:\n")
print(perfil_agrupado)

------------------------------------------------------------
 EXECUTANDO O MODELO DE REGRESSÃO REVISADO (SEM 'in_gratuito')
------------------------------------------------------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:                 qt_ing   No. Observations:                  997
Model:                            GLM   Df Residuals:                      986
Model Family:        NegativeBinomial   Df Model:                           10
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3662.3
Date:                qui, 18 set 2025   Deviance:                       1198.0
Time:                        10:05:26   Pearson chi2:                 1.07e+03
No. Iterations:                    29   Pseudo R-squ. (CS):             0.9124
Covariance Type:            nonrobust                                         
                         

/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [9]:
# Verifique a relação entre modalidade e categoria administrativa
print("Cruzamento: Modalidade vs. Categoria Administrativa")
print(pd.crosstab(df_modelo['tp_modalidade_ensino'], df_modelo['tp_categoria_administrativa']))

print("\n" + "---"*10 + "\n")

# Verifique a relação entre modalidade e grau acadêmico
print("Cruzamento: Modalidade vs. Grau Acadêmico")
print(pd.crosstab(df_modelo['tp_modalidade_ensino'], df_modelo['tp_grau_academico']))

Cruzamento: Modalidade vs. Categoria Administrativa
tp_categoria_administrativa  Privada com fins lucrativos  \
tp_modalidade_ensino                                       
Curso a distância                                    196   
Presencial                                           389   

tp_categoria_administrativa  Privada sem fins lucrativos  Pública Estadual  \
tp_modalidade_ensino                                                         
Curso a distância                                     83                 0   
Presencial                                           182                11   

tp_categoria_administrativa  Pública Federal  
tp_modalidade_ensino                          
Curso a distância                          7  
Presencial                               129  

------------------------------

Cruzamento: Modalidade vs. Grau Acadêmico
tp_grau_academico     Bacharelado  Licenciatura  Não aplicável  Tecnológico
tp_modalidade_ensino                                   

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# --- ETAPA 1: PREPARAÇÃO E ENGENHARIA DE ATRIBUTOS ---
# Esta seção prepara o DataFrame para a modelagem.

# Assuma que seu DataFrame original, já filtrado para a RIDE-DF, se chama 'df'.
# Ex: df = pd.read_csv('seu_censo_ride.csv')

# Tratamento inicial de valores infinitos que podem ter surgido em cálculos anteriores
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Criação de denominadores seguros para evitar erros de divisão por zero
df['qt_doc_exe_safe'] = df['qt_doc_exe'].replace(0, 1)
df['qt_vg_nova_safe'] = df['qt_vg_nova'].replace(0, 1)

# Criação das variáveis explicativas que serão usadas no modelo
df['prop_doc_doutores'] = df['qt_doc_ex_dout'] / df['qt_doc_exe_safe']
df['candidato_vaga'] = df['qt_insc_vg_nova'] / df['qt_vg_nova_safe']

# Filtrar o DataFrame para a análise (ex: cursos que tiveram pelo menos 1 inscrito)
# O .copy() é importante para evitar avisos de SettingWithCopyWarning
df_modelo = df[df['qt_insc_vg_nova'] > 0].copy()


# --- ETAPA 2: CORREÇÃO DA MULTICOLINEARIDADE ---
# Esta é a etapa crucial que resolve a instabilidade do modelo.

print("---" * 20)
print("CORRIGINDO O MODELO: Agrupando categorias 'Pública Estadual' e 'Pública Federal'")
print("---" * 20)

# Criar uma nova coluna 'categoria_admin_agrupada' para resolver a separação perfeita.
# A categoria 'Pública Estadual' é combinada com 'Pública Federal'.
df_modelo['categoria_admin_agrupada'] = df_modelo['tp_categoria_administrativa'].replace({
    'Pública Estadual': 'Pública',
    'Pública Federal': 'Pública'
})

# Você pode descomentar a linha abaixo para verificar se a nova categoria foi criada corretamente
# print("Distribuição da nova categoria administrativa:\n", df_modelo['categoria_admin_agrupada'].value_counts())


# --- ETAPA 3: CONSTRUÇÃO E TREINAMENTO DO MODELO FINAL ---
# Com os dados corrigidos, esta seção define, treina e exibe o resultado do modelo final.

# A fórmula final usa a nova variável 'categoria_admin_agrupada'
formula_final = """
    qt_ing ~ 
    candidato_vaga +
    prop_doc_doutores +
    in_capital_ies +
    C(tp_modalidade_ensino) +
    C(categoria_admin_agrupada) +
    C(tp_grau_academico)
"""

# Criação e treinamento do modelo final de Regressão Binomial Negativa
modelo_final = smf.glm(formula=formula_final, 
                       data=df_modelo, 
                       family=sm.families.NegativeBinomial()).fit()


# --- ETAPA 4: EXIBIÇÃO DO RESULTADO FINAL E ESTÁVEL ---
print("\nRESULTADO DO MODELO FINAL E ESTÁVEL:\n")
print(modelo_final.summary())

------------------------------------------------------------
CORRIGINDO O MODELO: Agrupando categorias 'Pública Estadual' e 'Pública Federal'
------------------------------------------------------------

RESULTADO DO MODELO FINAL E ESTÁVEL:

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 qt_ing   No. Observations:                  997
Model:                            GLM   Df Residuals:                      987
Model Family:        NegativeBinomial   Df Model:                            9
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3662.9
Date:                qui, 18 set 2025   Deviance:                       1199.2
Time:                        10:11:54   Pearson chi2:                 1.07e+03
No. Iterations:                    29   Pseudo R-squ. (CS):             0.9123
Covariance Type:            nonrobust          

/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# --- ETAPAS 1 E 2: PREPARAÇÃO E CORREÇÃO (Mantidas) ---
# Assumindo que 'df_modelo' já foi criado e a coluna 'categoria_admin_agrupada' também.
# Se estiver rodando do zero, execute o código completo anterior para criar este DataFrame.
df_final = df_modelo.copy()
df_final['categoria_admin_agrupada'] = df_final['tp_categoria_administrativa'].replace({
    'Pública Estadual': 'Pública',
    'Pública Federal': 'Pública'
})


# --- ETAPA 3: MODELO FINAL COM TROCA DE REFERÊNCIA ---
print("---" * 20)
print("EXECUTANDO MODELO FINAL COM TROCA DE CATEGORIA DE REFERÊNCIA")
print("---" * 20)

# Fórmula final com a alteração crucial:
# C(tp_modalidade_ensino, Treatment(reference='Presencial'))
# Isso força 'Presencial' a ser a base de comparação.
formula_com_referencia = """
    qt_ing ~ 
    candidato_vaga +
    prop_doc_doutores +
    in_capital_ies +
    C(tp_modalidade_ensino, Treatment(reference='Presencial')) +
    C(categoria_admin_agrupada) +
    C(tp_grau_academico)
"""

# Criação e treinamento do modelo
modelo_com_referencia = smf.glm(formula=formula_com_referencia, 
                                data=df_final, 
                                family=sm.families.NegativeBinomial()).fit()


# --- ETAPA 4: EXIBIÇÃO DO RESULTADO ---
print("\nRESULTADO DO MODELO COM AJUSTE DE REFERÊNCIA:\n")
print(modelo_com_referencia.summary())

------------------------------------------------------------
EXECUTANDO MODELO FINAL COM TROCA DE CATEGORIA DE REFERÊNCIA
------------------------------------------------------------

RESULTADO DO MODELO COM AJUSTE DE REFERÊNCIA:

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 qt_ing   No. Observations:                  997
Model:                            GLM   Df Residuals:                      987
Model Family:        NegativeBinomial   Df Model:                            9
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3662.9
Date:                qui, 18 set 2025   Deviance:                       1199.2
Time:                        10:13:45   Pearson chi2:                 1.07e+03
No. Iterations:                    29   Pseudo R-squ. (CS):             0.9123
Covariance Type:            nonrobust                     

/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# --- Assumindo que 'df_final' já existe com a categoria 'categoria_admin_agrupada' ---
# Se estiver rodando do zero, execute o código de uma das respostas anteriores para criar este DataFrame.

# --- ETAPA 1: MODELO FINAL SEM A MODALIDADE DE ENSINO ---
print("---" * 20)
print("EXECUTANDO O MODELO FINAL SEM A VARIÁVEL 'tp_modalidade_ensino'")
print("---" * 20)

# Fórmula final sem a variável que causa instabilidade.
# Este modelo irá estimar o efeito dos outros fatores de forma robusta.
formula_final_sem_modalidade = """
    qt_ing ~ 
    candidato_vaga +
    prop_doc_doutores +
    in_capital_ies +
    C(categoria_admin_agrupada) +
    C(tp_grau_academico)
"""

# Criação e treinamento do modelo
modelo_definitivo = smf.glm(formula=formula_final_sem_modalidade, 
                            data=df_final, 
                            family=sm.families.NegativeBinomial()).fit()


# --- ETAPA 2: EXIBIÇÃO DO RESULTADO DO MODELO DEFINITIVO ---
print("\nRESULTADO DO MODELO DEFINITIVO E ESTÁVEL:\n")
print(modelo_definitivo.summary())


# --- ETAPA 3: ANÁLISE DESCRITIVA DA MODALIDADE DE ENSINO ---
print("\n\n" + "---" * 20)
print("ANÁLISE DESCRITIVA SEPARADA PARA MODALIDADE DE ENSINO")
print("---" * 20)

# Calcular a média e a mediana de ingressantes por modalidade
analise_modalidade = df_final.groupby('tp_modalidade_ensino')['qt_ing'].agg(['mean', 'median', 'sum', 'count'])
print("\nEstatísticas de ingressantes por modalidade:\n")
print(analise_modalidade)

------------------------------------------------------------
EXECUTANDO O MODELO FINAL SEM A VARIÁVEL 'tp_modalidade_ensino'
------------------------------------------------------------

RESULTADO DO MODELO DEFINITIVO E ESTÁVEL:

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 qt_ing   No. Observations:                  997
Model:                            GLM   Df Residuals:                      988
Model Family:        NegativeBinomial   Df Model:                            8
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -4687.5
Date:                qui, 18 set 2025   Deviance:                       3248.4
Time:                        10:15:08   Pearson chi2:                 2.98e+03
No. Iterations:                    36   Pseudo R-squ. (CS):             0.3149
Covariance Type:            nonrobust                      

/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [19]:
# export df to csv
df.to_csv('censo_ride_df_completo.csv', index=False)

In [21]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

formula = """
qt_ing ~ C(tp_rede)
       + C(tp_categoria_administrativa)
       + C(tp_organizacao_academica)
       + C(tp_modalidade_ensino)
       + share_fem
       + share_noturno
       + share_ead
       + n_cursos
       + in_gratuito_bin
"""

modelo_poisson = smf.glm(formula=formula, data=df,
                         family=sm.families.Poisson()).fit()

print(modelo_poisson.summary())


PatsyError: Error evaluating factor: NameError: name 'share_ead' is not defined
    qt_ing ~ C(tp_rede)        + C(tp_categoria_administrativa)        + C(tp_organizacao_academica)        + C(tp_modalidade_ensino)        + share_fem        + share_noturno        + share_ead        + n_cursos        + in_gratuito_bin
                                                                                                                                                                                         ^^^^^^^^^

In [22]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# --- INÍCIO DO CÓDIGO DE ANÁLISE DIRETA ---
# O código abaixo assume que o DataFrame 'df' já existe e está carregado no seu ambiente.

# 1. CRIAR UMA CÓPIA DE TRABALHO PARA NÃO ALTERAR O DATAFRAME ORIGINAL
df_analysis = df.copy()

# 2. PREPARAÇÃO E ENGENHARIA DE ATRIBUTOS
df_analysis.replace([np.inf, -np.inf], np.nan, inplace=True)
df_analysis['qt_doc_exe_safe'] = df_analysis['qt_doc_exe'].replace(0, 1)
df_analysis['qt_vg_nova_safe'] = df_analysis['qt_vg_nova'].replace(0, 1)
df_analysis['prop_doc_doutores'] = (df_analysis['qt_doc_ex_dout'] / df_analysis['qt_doc_exe_safe']).fillna(0)
df_analysis['candidato_vaga'] = (df_analysis['qt_insc_vg_nova'] / df_analysis['qt_vg_nova_safe']).fillna(0)

# 3. APLICAÇÃO DO FILTRO CORRETO
# Filtra apenas por cursos que tiveram de fato ingressantes (> 0).
df_modelo = df_analysis[df_analysis['qt_ing'] > 0].copy()

# 4. AGRUPAMENTO DE CATEGORIAS PARA CORREÇÃO DO MODELO
df_modelo['categoria_admin_agrupada'] = df_modelo['tp_categoria_administrativa'].replace({
    'Pública Estadual': 'Pública',
    'Pública Federal': 'Pública'
})

# 5. DEFINIÇÃO DA FÓRMULA E EXECUÇÃO DO MODELO DE REGRESSÃO
formula_corrigida = """
    qt_ing ~ 
    candidato_vaga +
    prop_doc_doutores +
    in_capital_ies +
    C(categoria_admin_agrupada) +
    C(tp_grau_academico)
"""
modelo_corrigido = smf.glm(
    formula=formula_corrigida, 
    data=df_modelo, 
    family=sm.families.NegativeBinomial()
).fit()

# 6. EXECUÇÃO DA ANÁLISE DESCRITIVA PARA MODALIDADE DE ENSINO
analise_modalidade = df_modelo.groupby('tp_modalidade_ensino')['qt_ing'].agg(['mean', 'median', 'sum', 'count'])

# --- 7. EXIBIÇÃO DOS RESULTADOS ---
print("="*80)
print(" " * 20 + "RESULTADOS DO MODELO DE REGRESSÃO FINAL")
print("="*80)
print(modelo_corrigido.summary())

print("\n\n" + "="*80)
print(" " * 18 + "ANÁLISE DESCRITIVA POR MODALIDADE DE ENSINO")
print("="*80)
# A função display é ideal para notebooks, mas print funciona em qualquer ambiente
try:
    display(analise_modalidade)
except NameError:
    print(analise_modalidade)

/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/varfuncs.py:265: RuntimeWarning: overflow encountered in square
  return p + self.alpha*p**2
/home/jobsr/.venv/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:143: RuntimeWarning: invalid value encountered in multiply
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))


ValueError: NaN, inf or invalid value detected in weights, estimation infeasible.

In [3]:
# MODELO BAYESIANO

import pymc as pm
import arviz as az


# Função de preparação
def preparar_dados(df: pd.DataFrame) -> pd.DataFrame:
    """Cria variáveis derivadas e prepara dados para modelagem NB."""
    df = df.copy()
    df.columns = [c.lower() for c in df.columns]
    # Total de docentes
    df["qt_doc_total_calc"] = (
        df["qt_doc_ex_grad"].fillna(0)
        + df["qt_doc_ex_esp"].fillna(0)
        + df["qt_doc_ex_mest"].fillna(0)
        + df["qt_doc_ex_dout"].fillna(0)
    )
    # Proporção de docentes avançados (mestres + doutores)
    df["prop_doc_avancado"] = (
        (df["qt_doc_ex_mest"].fillna(0) + df["qt_doc_ex_dout"].fillna(0))
        / df["qt_doc_total_calc"].replace(0, np.nan)
    )
    # Proporções de ingressantes
    df["prop_ing_pp"] = (df["qt_ing_preta"] + df["qt_ing_parda"]) / df["qt_ing"].replace(0, np.nan)
    df["prop_ing_financiados"] = (
        df["qt_ing_fies"] + df["qt_ing_prounii"] + df["qt_ing_prounip"]
    ) / df["qt_ing"].replace(0, np.nan)
    # Tratar NaN e limitar
    for col in ["prop_doc_avancado", "prop_ing_pp", "prop_ing_financiados"]:
        df[col] = df[col].fillna(0).clip(0, 1)
    # Remover linhas inválidas
    df_model = df[(df["qt_ing"].notnull()) & (df["qt_ing"] >= 0) & (df["qt_mat"] > 0)].copy()
    # Criar offset
    df_model["offset_log_qtmat"] = np.log(df_model["qt_mat"])
    return df_model
# Função do Modelo Bayesiano 
def ajustar_modelo_bayesiano(df_model):
    y = df_model["qt_ing"].values
    offset = df_model["offset_log_qtmat"].values
    # Criar dummies das variáveis categóricas
    X = pd.get_dummies(
        df_model[[
            "tp_rede",
            "tp_organizacao_academica",
            "tp_grau_academico",
            "tp_modalidade_ensino"
        ]],
        drop_first=True
    )
    # Variáveis contínuas
    X["qt_conc"] = df_model["qt_conc"]
    X["prop_doc_avancado"] = df_model["prop_doc_avancado"]
    X["prop_ing_pp"] = df_model["prop_ing_pp"]
    X["prop_ing_financiados"] = df_model["prop_ing_financiados"]
    # Guardar nomes das variáveis
    colnames = X.columns.tolist()
    X = X.fillna(0).astype(float).values
    n, k = X.shape
    with pm.Model() as model:
        # Priors
        beta = pm.Normal("beta", mu=0, sigma=2, shape=k)
        intercept = pm.Normal("intercept", mu=0, sigma=5)
        alpha = pm.Exponential("alpha", 1)
        # Previsão linear + offset
        mu = pm.math.exp(intercept + pm.math.dot(X, beta) + offset)
        # Verossimilhança
        y_obs = pm.NegativeBinomial("y_obs", mu=mu, alpha=alpha, observed=y)
        # Amostragem
        trace = pm.sample(2000, tune=1000, chains=4, target_accept=0.95, random_seed=42)

    # salvar o trace em NetCDF
    az.to_netcdf(trace, "modelo_bayesiano_trace.nc")
    # salvar os resultados resumidos em CSV
    resultados = tabela_resultados(trace, colnames)
    resultados.to_csv("resultados_bayes.csv")
    
    return model, trace, colnames

def tabela_resultados(trace, colnames, hdi_prob=0.94):
    summary = az.summary(trace, hdi_prob=hdi_prob)
    # Extrair apenas os betas
    betas = summary.loc[[f"beta[{i}]" for i in range(len(colnames))]].copy()
    betas["variável"] = colnames
    betas = betas.set_index("variável")
    # Adicionar intercepto e alpha
    intercept = summary.loc["intercept"].to_frame().T
    intercept.index = ["Intercepto"]
    alpha = summary.loc["alpha"].to_frame().T
    alpha.index = ["Alpha (dispersão)"]
    resultados = pd.concat([betas, intercept, alpha], axis=0)
    return resultados

def forest_plot_bayes(resultados, hdi_prob=0.94):
    df_plot = resultados.copy()
    df_plot = df_plot.reset_index()
    df_plot = df_plot.rename(columns={
        "index": "Variável",
        "mean": "Média Posterior",
        f"hdi_{int((1-hdi_prob)/2*100)}%": "HDI_low",
        f"hdi_{int((1+hdi_prob)/2*100)}%": "HDI_high"
    })
    fig = px.scatter(
        df_plot,
        x="Média Posterior",
        y="Variável",
        error_x=df_plot["Média Posterior"] - df_plot["HDI_low"],
        error_x_minus=df_plot["HDI_high"] - df_plot["Média Posterior"],
        labels={"Média Posterior": "Coeficiente (escala log)", "Variável": "Variáveis"},
        size=[8]*len(df_plot),
        color=["Coeficiente"]*len(df_plot),
        symbol=["circle"]*len(df_plot)
    )
    fig.update_layout(
        title=f"Forest Plot Bayesiano (HDI {int(hdi_prob*100)}%)",
        xaxis_title="Efeito (log-razão de taxa)",
        xaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor="red"),
        height=700
    )
    return fig



In [5]:
## Executa
# Preparar dados
df_model = preparar_dados(df)
# Ajustar modelo bayesiano
model_bayes, trace_bayes, colnames_bayes = ajustar_modelo_bayesiano(df_model)

# salvar o trace em NetCDF
az.to_netcdf(trace_bayes, "modelo_bayesiano_trace.nc")
# salvar os resultados resumidos em CSV
resultados_bayes = tabela_resultados(trace_bayes, colnames_bayes)
resultados_bayes.to_csv("resultados_bayes.csv")

# Visualizar resultados
resultados_bayes


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, intercept, alpha]


Output()

: 

: 